In [4]:
#library to extract subtitles
%pip install youtube_transcript_api

In [1]:
#code needed to use colab
#if you're using juptyer notebook then skip this line
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
#cd used in colab
%cd "gdrive/My Drive/Colab Notebooks"
#%cd ..
!ls

/content/gdrive/My Drive/Colab Notebooks
 20200504-193926_joe_biden_nlp_subs_clean_expanded.csv
'20200625-155513_gun_control (1).csv'
 20200625-155513_gun_control_nlp.csv
 black_lives_nlp_subs_clean_expanded.csv
 Extract_subtitles.ipynb
 nlp_EDA_v3.ipynb
 nlp_EDA_v4.ipynb
 nlp_model_training_v2.ipynb
 pandemic_nlp_subs_clean_expanded.csv


In [5]:
import pandas as pd
import numpy as np
from youtube_transcript_api import YouTubeTranscriptApi
from tqdm import tqdm

# Extract Subtitles

In [21]:
#import csv
df = pd.read_csv('20200625-155513_gun_control_nlp.csv', index_col=0)
df.head(2)

,Bias_num,title,description,channel,id
1,0,"Richard Dawkins on Islam, Jews, science and th...",Evan Davis speaks to Richard Dawkins. ** Subsc...,BBC Newsnight,YAV_0s1c2V4
2,0,What would a second wave look like? | CNBC Exp...,Past pandemics have shown that multiple waves ...,CNBC International,Hc2kSVBuDns


In [22]:
#length of dataset
print(f'Length of dataset is: {len(df)}')

#percentage of videos that are right
pr_right = round(df['Bias_num'].mean()*100,2)
print(f'Percentage of videos that are right: {pr_right} %')

Length of dataset is: 57
Percentage of videos that are right: 10.53 %


In [23]:
#loop to extract subtitles

tttt = []

with tqdm(total=len(df['id'])) as pbar:  
    for idx in df['id']:
        try:
            t = YouTubeTranscriptApi.get_transcript(idx)
        except:
            tttt.append([idx, np.nan]) #add NaN to videos that do not have subtitles
            continue
        tt = [i["text"] for i in t]
        ttt = [' '.join(tt)]
        tttt.append([idx, ttt])
        pbar.update(1)

df_sub = pd.DataFrame(tttt, columns=['video_id', 'subtitles'])
df_sub.head()

 88%|████████▊ | 50/57 [00:29<00:04,  1.70it/s]


,video_id,subtitles
0,YAV_0s1c2V4,[Dawkins is famous as the author of The Selfis...
1,Hc2kSVBuDns,[Of the last five pandemics the world has face...
2,b0BC3F0kqtw,"[Oh, thank you Fairey Thank you Thank you. Ver..."
3,SYPZwZud_PA,[I'm the attorney first CFI and I'm the person...
4,xU-3Zdei1N0,[Susi's had a complex and thing because I hear...


In [24]:
#merge subtitles with dataframe containing video id, channel, etc. 
df2 = df.merge(df_sub, left_on='id', right_on='video_id')

In [25]:
#i checked a few nan subtitles and they don't have subtitles. 
df2[df2['subtitles'].isna()].head()

,Bias_num,title,description,channel,id,video_id,subtitles
10,0,"""""Obama out:"""" President Barack Obama's hilari...","As President Obama gets set to leave office, t...",Global News,NxFkEj7KPC0,NxFkEj7KPC0,NaN
11,1,Jim Jefferies on how he got Brad Pitt to do th...,Catch “The Jim Jefferies Show” Tuesday’s at 10...,KTLA 5,oECEqyLywgE,oECEqyLywgE,NaN
17,0,Mark Cuban | Full Q&A | Oxford Union,SUBSCRIBE for more speakers ► http://is.gd/Oxf...,OxfordUnion,ii36plwJAnc,ii36plwJAnc,NaN
18,0,Bill Maher | Full Q&A | Oxford Union,*Contains Strong Language*SUBSCRIBE for more s...,OxfordUnion,MPb1VNt2EOo,MPb1VNt2EOo,NaN
24,0,We're Doing Elections Wrong | Patriot Act with...,How did the 2020 presidential election in Amer...,Patriot Act,MykMQfmLIro,MykMQfmLIro,NaN


In [26]:
#drop column
df2.drop('video_id', inplace=True, axis=1)

In [27]:
#export csv
df2.to_csv("20200625-155513_gun_control_nlp_subs.csv")

# Subtitles cleaning

In [28]:
# load train_cleaned.csv
filename = "20200504-193926_joe_biden_nlp_subs.csv"
df = pd.read_csv(filename, usecols=['Bias_num', 'subtitles','channel'])

#copy to not destroy original dataset
df_channel = df.copy()

#create new column text. easier in the analysis
df['text'] = df['subtitles']

#drop subtitles and channel
df.drop(['subtitles'], axis=1, inplace=True)

In [29]:
#which videos have NaN as subtitle
df_nan = df[df['text'].isna()]

#number of videos with no subtitles
num_nan = len(df_nan)
print(f'Number of videos that do NOT have subtitles: {num_nan}')

print()

#displays channels with missing subtitles
channel = df_nan.groupby(["channel"])['Bias_num'].count().sort_values(ascending=False)
print(channel[:5])

print()

#display distribution of bias with missing subtitles
bias = df_nan.groupby('Bias_num')['channel'].count()
print(bias)

Number of videos that do NOT have subtitles: 7

channel
OxfordUnion              2
euronews (in English)    1
Vox                      1
Patriot Act              1
KTLA 5                   1
Name: Bias_num, dtype: int64

Bias_num
0    5
1    2
Name: channel, dtype: int64


In [30]:
#drop channel
df.drop(['channel'], axis=1, inplace=True)

#drop NaNs
df.dropna(inplace=True)

#remove brackets [] and quotations marks
df['text'] = df['text'].str[2:-2]

df.head(2)

,Bias_num,text
0,0,Dawkins is famous as the author of The Selfish...
1,0,Of the last five pandemics the world has faced...


In [31]:
#average right wing videos
df['Bias_num'].mean()

0.08

In [32]:
#split channel string to remove each element from subtitles

#remove (in English) since the brackets cause problems with replacing values
df_channel['channel'] = df_channel['channel'].replace("(in English)", '', regex=True)

#list all unique channels
unique_channel = df_channel['channel'].unique()

#split channel name by whitespace
unique_channel_split = [i.split(' ') for i in unique_channel]

#display first row
unique_channel_split[0]

['BBC', 'Newsnight']

In [33]:
#loop to remove each single word in channel name
#doing this to remove any possibility that the NLP model learns channel to predict bias
for channel in unique_channel_split:
    for element in channel:
        df['text'].replace(element,
                           '',
                           inplace=True,
                           regex=True)

In [34]:
#the text [Music] appears in subtitles when music is playing. 
#these values mess up the analysis in the end with 3-gram, 2-gram
to_replace = ['Music', 'music', 'Applause', 'applause', 'Laughter', 'laughter', '♪', "'", "\n", "\\\92", "\\\96", 
              "\\\\", 'AUDIENCE LAUGHING', 'AUDIENCE APPLAUDING', 'AUDIENCE GASPS', "NARRATOR"]

df['text'].replace(to_replace, '', inplace=True, regex=True)  

In [35]:
df.to_csv("20200625-155513_gun_control_nlp_subs_clean.csv")

## Split subtitles after 1'000 characters

We split at 1'000 characters to make our dataset larger. I (Rolf) thinks that having more rows is more important for the model training than few but long rows. I think that 1'000 characters is a good cut-off. This is an arbitrary number. 

In [36]:
# load train_cleaned.csv
filename = "20200625-155513_gun_control_nlp_subs_clean.csv"
df_clean = pd.read_csv(filename, index_col=0)

df_clean.head()

,Bias_num,text
0,0,Dawkins is famous as the author of Selfish Ge...
1,0,Of the last five pandemics the world has faced...
2,0,"Oh, thank you Fairey Thank you Thank you. Very..."
3,0,Im the attorney first CFI and Im the person pr...
4,0,Susis had a complex and thing because I heard ...


In [37]:
#df_clean has this many rows

print('df_clean has this many rows: ' + str(len(df_clean)))

df_clean has this many rows: 50


In [38]:
#function to split text into columns after 1'000 characters
def chunks(s, n):
    """Produce `n`-character chunks from `s`."""
    for start in range(0, len(s), n):
        yield s[start:start+n]

In [39]:
#empty lists to keep info
sub = []
bias = []

#loop to split at 1'000 characters
for index, row in df.iterrows():
    for chunk in chunks(row['text'], 1000):
        sub.append(chunk)
        bias.append(row['Bias_num'])

In [40]:
#create a dataframe from sub and bias list
df_sub = pd.DataFrame(list(zip(bias, sub)), columns=['bias_num', 'text'])

length = len(df_sub)
print(f"Number of rows after splitting at 1'000 characters: {length}")

percentage = round(df_sub['bias_num'].mean()*100, 2)
print(f"Percentage of rows that are right wing: {percentage}%")

df_sub.head()

Number of rows after splitting at 1'000 characters: 856
Percentage of rows that are right wing: 2.92%


,bias_num,text
0,0,Dawkins is famous as the author of Selfish Ge...
1,0,ink the rules of the game should be between a ...
2,0,nt matter what I feel nobody else should abide...
3,0,the spirit or critical thinking its a tricky ...
4,0,of that accusation and do you if you like rev...


In [41]:
#loop to calculate average length of text per row

length = []

for index, row in df_sub.iterrows():
    length.append(len(row['text']))
    
avg_length = round(sum(length)/len(length),2)
print(f"Average character length of text column per row: {avg_length}")

Average character length of text column per row: 973.28


In [42]:
df_sub.to_csv("20200625-155513_gun_control_nlp_subs_clean_expanded.csv")